# Fine-tuning BERT French

In [1]:
# !pip install -q transformers datasets

## Load dataset

Next, let's download a multi-label text classification dataset [MultiEURLEX](https://huggingface.co/datasets/multi_eurlex)


In [1]:
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
from datasets import load_dataset

dataset = load_dataset('multi_eurlex', 'fr')

Reusing dataset multi_eurlex (/home/davo/.cache/huggingface/datasets/multi_eurlex/fr/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)


  0%|          | 0/3 [00:00<?, ?it/s]

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [3]:

dataset = dataset.rename_column("labels", "old_labels")

In [4]:

dataset

DatasetDict({
    train: Dataset({
        features: ['celex_id', 'text', 'old_labels'],
        num_rows: 55000
    })
    test: Dataset({
        features: ['celex_id', 'text', 'old_labels'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['celex_id', 'text', 'old_labels'],
        num_rows: 5000
    })
})

Let's check the first example of the training split:

In [5]:
example = dataset['train'][0]
example

{'celex_id': '32006D0213',
 'text': "DÉCISION DE LA COMMISSION\ndu 6 mars 2006\nétablissant la classification des caractéristiques de réaction au feu de certains produits de construction pour ce qui concerne les sols en bois et les lambris et revêtements muraux extérieurs en bois massif\n[notifiée sous le numéro C(2006) 655]\n(Texte présentant de l'intérêt pour l'EEE)\n(2006/213/CE)\nLA COMMISSION DES COMMUNAUTÉS EUROPÉENNES,\nVu le traité instituant la Communauté européenne,\nVu la directive 89/106/CEE du Conseil du 21 décembre 1988 relative au rapprochement des dispositions législatives, réglementaires et administratives des États membres concernant les produits de construction (1), et notamment son article 20, paragraphe 2,\nConsidérant ce qui suit:\n(1)\nLa directive 89/106/CEE considère que, afin de tenir compte des différences de niveau de protection existant à l’échelon national, régional ou local, il peut être nécessaire, pour chaque exigence essentielle, d’établir des classes 

The dataset consists of tweets, labeled with one or more emotions. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [6]:
labels = ['social questions',
 'industry',
 'finance',
 'trade',
 'business and competition',
 'international relations',
 'agriculture, forestry and fisheries',
 'production, technology and research',
 'transport',
 'employment and working conditions',
 'politics',
 'law',
 'education and communications',
 'international organisations',
 'energy',
 'EUROPEAN UNION',
 'science',
 'agri-foodstuffs',
 'geography',
 'economics',
 'environment']
id2label = {0: 'social questions',
 1: 'industry',
 2: 'finance',
 3: 'trade',
 4: 'business and competition',
 5: 'international relations',
 6: 'agriculture, forestry and fisheries',
 7: 'production, technology and research',
 8: 'transport',
 9: 'employment and working conditions',
 10: 'politics',
 11: 'law',
 12: 'education and communications',
 13: 'international organisations',
 14: 'energy',
 15: 'EUROPEAN UNION',
 16: 'science',
 17: 'agri-foodstuffs',
 18: 'geography',
 19: 'economics',
 20: 'environment'}
label2id = {'social questions': 0,
 'industry': 1,
 'finance': 2,
 'trade': 3,
 'business and competition': 4,
 'international relations': 5,
 'agriculture, forestry and fisheries': 6,
 'production, technology and research': 7,
 'transport': 8,
 'employment and working conditions': 9,
 'politics': 10,
 'law': 11,
 'education and communications': 12,
 'international organisations': 13,
 'energy': 14,
 'EUROPEAN UNION': 15,
 'science': 16,
 'agri-foodstuffs': 17,
 'geography': 18,
 'economics': 19,
 'environment': 20}
labels

['social questions',
 'industry',
 'finance',
 'trade',
 'business and competition',
 'international relations',
 'agriculture, forestry and fisheries',
 'production, technology and research',
 'transport',
 'employment and working conditions',
 'politics',
 'law',
 'education and communications',
 'international organisations',
 'energy',
 'EUROPEAN UNION',
 'science',
 'agri-foodstuffs',
 'geography',
 'economics',
 'environment']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [7]:
def numbers_to_classes(l=[0,10,20]):
  zero_cl = [0.0] * 21
  for i in l:
    zero_cl[i] = 1.0
  
  return np.array(zero_cl, dtype=np.float)

In [8]:
numbers_to_classes(l=[0,10,20])

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1.])

In [9]:
from transformers import AutoTokenizer
import numpy as np
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=300)
  # add labels
  labels_matrix = []
  for r in examples["old_labels"]:
    labels_matrix.append(np.array(numbers_to_classes(r), dtype=float))
  # print(labels_matrix)
  encoding["labels"] = np.array(labels_matrix, dtype=float)
  
  return encoding

In [10]:
encoded_dataset = dataset.map(preprocess_data, batched=True)

Parameter 'function'=<function preprocess_data at 0x7fbfc1109200> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/55 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [11]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['celex_id', 'text', 'old_labels', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [12]:
tokenizer.decode(example['input_ids'])

"[CLS] decision de la commission du 6 mars 2006 etablissant la classification des caracteristiques de reaction au feu de certains produits de construction pour ce qui concerne les sols en bois et les lambris et revetements muraux exterieurs en bois massif [ notifiee sous le numero c ( 2006 ) 655 ] ( texte presentant de l'interet pour l'eee ) ( 2006 / 213 / ce ) la commission des communautes europeennes, vu le traite instituant la communaute europeenne, vu la directive 89 / 106 / cee du conseil du 21 decembre 1988 relative au rapprochement des dispositions legislatives, reglementaires et administratives des etats membres concernant les produits de construction ( 1 ), et notamment son article 20, paragraphe 2, considerant ce qui suit : ( 1 ) la directive 89 / 106 / cee considere que, afin de tenir compte des differences de niveau de protection existant a l [UNK] echelon national, regional ou local, il peut etre necessaire, pour chaque exigence essentielle, d [UNK] etablir des classes de 

In [13]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['social questions',
 'industry',
 'trade',
 'production, technology and research',
 'environment']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [14]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-multilingual-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [15]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [16]:
batch_size = 16
metric_name = "f1"

In [17]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-french",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [18]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [19]:
encoded_dataset['train']['labels'][0]

tensor([1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1.])

In [20]:

encoded_dataset['train']['labels']

tensor([[1., 1., 0.,  ..., 0., 0., 1.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [21]:
encoded_dataset['train']['input_ids'][0]

tensor([  101, 15308, 10102, 10106, 13932, 10169,   127, 11362, 10231, 61684,
        50975, 10106, 10485, 10143, 25731, 10102, 25274, 10257, 25506, 10102,
        17492, 28575, 10102, 13154, 10343, 10630, 10379, 52089, 10152, 31680,
        10109, 16556, 10137, 10152, 51231, 12531, 10137, 58831, 50345, 10107,
        39721, 12204, 43311, 10107, 10109, 16556, 43835,   138, 69327, 66340,
        11520, 10130, 11855,   145,   113, 10231,   114, 49168,   140,   113,
        19703, 59627, 10102,   154,   112, 36719, 10343,   154,   112, 23329,
        10111,   114,   113, 10231,   120, 21232,   120, 10630,   114, 10106,
        13932, 10143, 66008, 65919,   117, 12310, 10130, 25269, 52119, 24422,
        11539, 10106, 18816, 29417,   117, 12310, 10106, 92695, 12844,   120,
        16141,   120, 10630, 10111, 10169, 16727, 10169, 10259, 14110, 10632,
        23350, 10257, 20786, 50028, 70473, 10143, 53909, 10107, 55478,   117,
        89590, 28666, 10137, 17376, 10107, 10143, 13371, 16294, 

In [22]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train']['labels'][0].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7026, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), logits=tensor([[-0.0917,  0.0497,  0.0852,  0.0350, -0.1104, -0.0564,  0.0985,  0.1197,
          0.0737,  0.1160,  0.0067,  0.0303,  0.0420,  0.1266, -0.0892, -0.0814,
          0.1627,  0.0551,  0.0049,  0.1038,  0.0967]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

Let's start training!

In [23]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/davo/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/davo/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/davo/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/davo/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


In [24]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: old_labels, celex_id, text. If old_labels, celex_id, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/davo/anaconda3/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 55000
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 51570
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as:

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.146700,0.180381,0.801885,0.863108,0.273200
2,0.123900,0.170703,0.816280,0.875694,0.287400
3,0.109100,0.171347,0.820302,0.878735,0.279600
4,0.096900,0.172468,0.819982,0.879797,0.300000
5,0.086400,0.176317,0.823362,0.881583,0.297400
6,0.075600,0.178507,0.822162,0.885190,0.284800
7,0.067800,0.184399,0.824527,0.885666,0.297400
8,0.061400,0.192329,0.827326,0.889268,0.303600
9,0.051200,0.200938,0.821519,0.884153,0.290000
10,0.046100,0.209489,0.822991,0.888141,0.301000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: old_labels, celex_id, text. If old_labels, celex_id, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to bert-finetuned-sem_eval-french/checkpoint-3438
Configuration saved in bert-finetuned-sem_eval-french/checkpoint-3438/config.json
Model weights saved in bert-finetuned-sem_eval-french/checkpoint-3438/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-french/checkpoint-3438/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-french/checkpoint-3438/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: old_labels, celex_id, text. If old_

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: old_labels, celex_id, text. If old_labels, celex_id, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to bert-finetuned-sem_eval-french/checkpoint-37818
Configuration saved in bert-finetuned-sem_eval-french/checkpoint-37818/config.json
Model weights saved in bert-finetuned-sem_eval-french/checkpoint-37818/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-french/checkpoint-37818/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-french/checkpoint-37818/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: old_labels, celex_id, text. If

TrainOutput(global_step=51570, training_loss=0.07078056117234037, metrics={'train_runtime': 20620.909, 'train_samples_per_second': 40.008, 'train_steps_per_second': 2.501, 'total_flos': 1.27209170385e+17, 'train_loss': 0.07078056117234037, 'epoch': 15.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [25]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: old_labels, celex_id, text. If old_labels, celex_id, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16


{'eval_loss': 0.1923290342092514,
 'eval_f1': 0.8273264896228522,
 'eval_roc_auc': 0.8892684407566105,
 'eval_accuracy': 0.3036,
 'eval_runtime': 37.0849,
 'eval_samples_per_second': 134.826,
 'eval_steps_per_second': 8.44,
 'epoch': 15.0}

## Inference

Let's test the model on a new sentence:

In [26]:
dataset['test']['text'][0]

"RÈGLEMENT (UE) No 1390/2013 DU CONSEIL\ndu 16 décembre 2013\nrelatif à la répartition des possibilités de pêche au titre du protocole entre l'Union européenne et l'Union des Comores fixant les possibilités de pêche et la contrepartie financière prévues par l'accord de partenariat dans le secteur de la pêche en vigueur entre les deux parties\nLE CONSEIL DE L'UNION EUROPÉENNE,\nvu le traité sur le fonctionnement de l'Union européenne, et notamment son article 43, paragraphe 3,\nvu la proposition de la Commission européenne,\nconsidérant ce qui suit:\n(1)\nLe 5 octobre 2006, le Conseil a approuvé l'accord de partenariat dans le secteur de la pêche entre la Communauté européenne et l'Union des Comores (ci-après dénommé «accord de partenariat») en adoptant le règlement (CE) no 1563/2006 (1).\n(2)\nL'Union européenne a négocié avec l'Union des Comores un nouveau protocole à l'accord de partenariat accordant aux navires de l'Union européenne des possibilités de pêche dans les eaux comorienne

### Below you can observe a few examples with their predicted outputs for Bert model and what actually was true label

In [27]:
def predict_labels(text, thres_prob=0.5):
    encoding = tokenizer(text, return_tensors="pt")
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = trainer.model(**encoding)
    logits = outputs.logits
    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= thres_prob)] = 1
    # turn predicted id's into actual label names
    predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]

    return predicted_labels

In [28]:
predicted_labels = predict_labels(text=dataset['test']['text'][0][:512])

In [29]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['international relations', 'agriculture, forestry and fisheries', 'EUROPEAN UNION', 'geography']


In [30]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][0])) if label == 1.0])

true labels>>> ['international relations', 'agriculture, forestry and fisheries', 'EUROPEAN UNION', 'geography']


### In the case above Bert model was able to pick up all true categories including 'EUROPEAN UNION'  category

In [31]:
predicted_labels = predict_labels(text=dataset['test']['text'][100][:512])

In [32]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['trade', 'agriculture, forestry and fisheries', 'agri-foodstuffs', 'geography']


In [33]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][100])) if label == 1.0])

true labels>>> ['trade', 'agriculture, forestry and fisheries', 'production, technology and research', 'agri-foodstuffs', 'geography']


### In the case above Bert model was able to pick up all true categories but missed 'production, technology and research'  category

In [34]:
predicted_labels = predict_labels(text=dataset['test']['text'][1000][:512])

In [35]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['international relations', 'EUROPEAN UNION', 'geography']


In [36]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][1000])) if label == 1.0])

true labels>>> ['trade', 'international relations', 'EUROPEAN UNION', 'geography']


### In the case above Bert model was able to pick up all true categories but also missed 'trade' category

In [37]:
predicted_labels = predict_labels(text=dataset['test']['text'][1000][:512], thres_prob=0.01)

In [38]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['trade', 'international relations', 'transport', 'law', 'education and communications', 'EUROPEAN UNION', 'geography', 'economics']


In [39]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][1000])) if label == 1.0])

true labels>>> ['trade', 'international relations', 'EUROPEAN UNION', 'geography']


### if we reduce threshold to 1% for accepting  a label as related to the class then we capture trade but mislabel transport, law, education and communications, economics

In [40]:
predicted_labels = predict_labels(text=dataset['test']['text'][1111][:512])

In [41]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['social questions', 'production, technology and research', 'agri-foodstuffs']


In [42]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][1111])) if label == 1.0])

true labels>>> ['trade', 'agri-foodstuffs']


## we don't capture trade  and mislabel social questions category and  'production, technology and research'

In [43]:
predicted_labels = predict_labels(text=dataset['test']['text'][2230][:512])

In [44]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['trade', 'agri-foodstuffs']


In [45]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][2230])) if label == 1.0])

true labels>>> ['trade', 'agri-foodstuffs']


### Based on output above we can observe that Bert returns also fully correct list of labels as well.

### Based on examples above it is clear thta BERT model learned to pick up majority of the categories but very often missed one category from the true category list.  Good part Bert picks up majority of the classes, bad it often misses one class

## Metrics that would be useful for multilabel classification are F1 score and AUC/ROC curve. We will stick to F1

In [46]:
{'eval_loss': 0.1923290342092514,
 'eval_f1': 0.8273264896228522,
 'eval_roc_auc': 0.8892684407566105,
 'eval_accuracy': 0.3036,
 'eval_runtime': 37.0849,
 'eval_samples_per_second': 134.826,
 'eval_steps_per_second': 8.44,
 'epoch': 15.0}

{'eval_loss': 0.1923290342092514,
 'eval_f1': 0.8273264896228522,
 'eval_roc_auc': 0.8892684407566105,
 'eval_accuracy': 0.3036,
 'eval_runtime': 37.0849,
 'eval_samples_per_second': 134.826,
 'eval_steps_per_second': 8.44,
 'epoch': 15.0}

## above we can observe performance measures of BERT model trained with 50 epochs for the test set with f1 score  82.7%

In [133]:
1/21

0.047619047619047616

## To compare with random classifier we have 21 categories so random classifier which always predicts same category should be able to obtain around 5% (equal distribution of categoris in datasets) accuracy while our model has 30.3% on test set.

#### Bert improvement of the model for French

To improve Bert model for Frech language which has lower performance metrics compared to English and Spanish Bert models we will tokenizers specifically fine tuned on Fench language as previously we have used bert-base-multilingual-uncased. With using this approach it is highly likely we get higher performance